#### Flask API

In [3]:
import os
flask_api_file = os.path.join(os.path.pardir, 'src', 'models', 'flask_api.py')

In [7]:
%%writefile $flask_api_file
from flask import Flask, request

app = Flask(__name__)

@app.route('/api', methods=['POST'])
def say_hello():
    data = request.get_json(force=True)
    name = data['name']
    return 'hello {}'.format(name)

if __name__ == '__main__':
    app.run(port=10001, debug=True)

Overwriting ../src/models/flask_api.py


In [8]:
import json
import requests

In [12]:
url = 'http://127.0.0.1:10001/api'
data = json.dumps({'name':'abhi'})
r = requests.post(url, data)
print r.text

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=10001): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x108902a10>: Failed to establish a new connection: [Errno 61] Connection refused',))

##### Machine Learning API using Flask.

##### Building API

In [10]:
machine_learning_api_script_file = os.path.join(os.path.pardir, 'src', 'models', 'machine_learning_api.py')


In [14]:
%%writefile machine_learning_api_script_file

from flask import Flask, request
import pandas as pd
import numpy as np
import json
import os

app = Flask(__name__)

# Load model and scaler files
model_path = os.path.join(os.path.pardir, os.path.pardir, 'models')
model_filepath = os.path.join(model_path, 'models', 'lr_model.pkl')
scaler_filepath = os.path.join(model_path, 'models', 'lr_scaler.pkl')

scaler = pickle.load(open(scaler_filepath))
model = pickle.load(open(model_filepath))

#columns
columns = ['Age', 'Fare', 'FamilySize', 
           'IsMother', 'IsMale', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_Z',
           'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Lady', 'Title_Master', 'Title_Miss', 'Title_Mr',
           'Title_Mrs', 'Title_Officer', 'Title_Sir', 'Fare_Bin_very_low', 'Fare_Bin_low', 'Fare_Bin_high',
           'Fare_Bin_very_high', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'AgeState_Adult', 'AgeState_Child'
    
]

@app.route('/api', methods=['POST'])
def make_prediction():
    data = json.dumps(request.get_json(force=True))
    df = pd.read_json(data)
    passenger_ids = df['PassengerId'].ravel()
    actuals = df['Survived'].ravel()
    X = df[columns].as_matrix().astype('float')
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    df_response = pd.Dataframe({
        'PassengerId':  passenger_ids,
        'Predicted': predictions,
        'Actual': actuals
    })
    return df_response.to_json()

if __name__ == '__main__':
    # host flask app at port 10001
    app.run(port=10001, debug=True)


Overwriting machine_learning_api_script_file


In [15]:
##### Invoking API using request
import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
train_df = pd.read_csv(train_file_path)

In [16]:
survived_passengers = train_df[train_df['Survived'] == 1][:5]

In [17]:
survived_passengers

,PassengerId,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_Bin_very_low,Fare_Bin_low,Fare_Bin_high,Fare_Bin_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,2,1,38.0,71.2833,2,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
2,3,1,26.0,7.9250,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,4,1,35.0,53.1000,2,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
8,9,1,27.0,11.1333,3,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
9,10,1,14.0,30.0708,2,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [18]:
import requests
def make_api_request(data):
    # url for api
    url = 'http://127.0.0.1:10001/api'
    # make post request
    r = requests.post(url, data)
    return r.json()

In [19]:
 make_api_request(survived_passengers.to_json())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=10001): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10b9be3d0>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [20]:
# compare results
result = make_api_request(train_df.to_json())
df_result = pd.read_json(json.dumps(result))
df_result.head()

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=10001): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10bc01590>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [21]:
import numpy as np
np.mean(df.result.Actual == df_result.Predicted)

NameError: name 'df' is not defined